## MULTI-LABEL IMAGE CLASSIFICATION WITH KERAS

### LOADING LIBRARIES

In [ ]:
!pip install pillow
!pip install sklearn

In [ ]:

import PIL
import PIL.Image
from sklearn.utils import shuffle
import pandas as pd

In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
import numpy as np
#from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Adam
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation

In [5]:

import glob,matplotlib as plt
%matplotlib inline

In [ ]:
#from keras.applications.vgg16 import VGG16, preprocess_input,decode_predictions

In [6]:
from keras.applications.xception import Xception,preprocess_input, decode_predictions

### LOADING DATA

In [7]:
train=pd.read_csv('./meta-data/train.csv')
test=pd.read_csv('./meta-data/test.csv')


In [8]:
img_width, img_height = 224, 224

In [9]:
train.shape,test.shape

((12600, 86), (5400, 1))

In [10]:

import cv2
def read_image(img_path,H,W):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (W,H)) # you can resize to  (128,128) or (256,256)
    return img


In [11]:
def save_model(model_final,filename):
    # serialize model to JSON
    from keras.models import model_from_json
    model_json = model_final.to_json()
    with open(filename+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model_final.save_weights(filename+".h5")
    print("Saved model to disk")
    
from keras.models import model_from_json
# load json and create model
def load_model(filename):
    json_file = open(filename+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model_final = model_from_json(loaded_model_json)
    # load weights into new model
    model_final.load_weights(filename+".h5")
    print("Loaded model from disk")
    return model_final

In [12]:
import gc
gc.collect()

15

In [13]:
%%time
image_lr=[]
train_img=[]
for i in list(train.Image_name):
    image_lr.append(read_image('./train_img/'+str(i),img_width, img_height))
train_img=np.array(image_lr,dtype=np.float32)
train_img=preprocess_input(train_img)

CPU times: user 3min 32s, sys: 3.86 s, total: 3min 36s
Wall time: 3min 36s


In [14]:
train_img.shape#,test_img.shape

(12600, 224, 224, 3)

In [15]:

# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
#rescale = 1./255,   
horizontal_flip = True,
zoom_range = 0.2,
fill_mode="nearest" ,   
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=20)

'''
test_datagen = ImageDataGenerator(
#rescale = 1./255,
horizontal_flip = True,
zoom_range = 0.2,
width_shift_range = 0.2,
height_shift_range=0.2,
rotation_range=20)
'''
gc.collect()


0

In [16]:
Y=train.iloc[:,1:]
Y.shape

(12600, 85)

In [ ]:
'''
l=int(train.shape[0]-(0.1*train.shape[0]))
l
x_train=train_img[:l,:,:,:]
y_train=Y[:l]
x_valid=train_img[l:,:,:,:]
y_valid=Y[l:]
x_train.shape,y_train.shape,x_valid.shape,y_valid.shape
'''

In [17]:
batch_size=256

In [18]:


train_generator = train_datagen.flow(train_img,Y,
                                    batch_size = batch_size,
                                    shuffle=True)
                                    #class_mode = "categorical")
    
'''
validation_generator = test_datagen.flow(x_valid,y_valid,
                                         batch_size = batch_size,
                                        shuffle=True)
                                        #class_mode = "categorical")

'''

'\nvalidation_generator = test_datagen.flow(x_valid,y_valid,\n                                         batch_size = batch_size,\n                                        shuffle=True)\n                                        #class_mode = "categorical")\n\n'

In [19]:
model = applications.Xception(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [20]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers:
    layer.trainable = False

In [21]:


#Adding custom Layers 
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(128, activation="relu")(x)
predictions = Dense(85, activation="sigmoid")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if sys.path[0] == '':


In [22]:
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 1

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [23]:
adam = Adam(lr=1e-2)
model_final.compile(optimizer=adam, loss='binary_crossentropy', metrics=[fbeta])

In [24]:
#validation_data=validation_generator, validation_steps=x_valid.shape[0]//batch_size, 
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=2,
                          shuffle=True)

Epoch 1/2
49/49 [==============================] - 148s 3s/step - loss: 0.4202 - fbeta: 0.7212
Epoch 2/2
49/49 [==============================] - 144s 3s/step - loss: 0.2510 - fbeta: 0.8482


In [25]:
# Train the model 
K.set_value(adam.lr, 0.01)
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=2,
                          shuffle=True)
#model_final.fit(train_img, Y,batch_size=256,epochs=3,shuffle=True,verbose=1)

Epoch 1/2
49/49 [==============================] - 146s 3s/step - loss: 0.2142 - fbeta: 0.8713
Epoch 2/2
49/49 [==============================] - 144s 3s/step - loss: 0.1978 - fbeta: 0.8839


In [26]:
# Train the model 
K.set_value(adam.lr, 0.001)
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=4,
                          shuffle=True)
#model_final.fit(train_img, Y,batch_size=256,epochs=3,shuffle=True,verbose=1)

Epoch 1/4
49/49 [==============================] - 146s 3s/step - loss: 0.1837 - fbeta: 0.8942
Epoch 2/4
49/49 [==============================] - 144s 3s/step - loss: 0.1762 - fbeta: 0.8987
Epoch 3/4
49/49 [==============================] - 143s 3s/step - loss: 0.1705 - fbeta: 0.9017
Epoch 4/4
49/49 [==============================] - 144s 3s/step - loss: 0.1686 - fbeta: 0.9028


In [34]:
# Train the model 
K.set_value(adam.lr, 0.0001)
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=2,
                          shuffle=True)

Epoch 1/2
49/49 [==============================] - 146s 3s/step - loss: 0.1684 - fbeta: 0.9033
Epoch 2/2
49/49 [==============================] - 144s 3s/step - loss: 0.1637 - fbeta: 0.9061


In [35]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model_final.layers[:100]:
    layer.trainable = False
for layer in model_final.layers[100:]:
    layer.trainable = True

In [36]:
adam = Adam(lr=1e-4)
model_final.compile(optimizer=adam, loss='binary_crossentropy', metrics=[fbeta])
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=4,
                          shuffle=True)

Epoch 1/4
49/49 [==============================] - 192s 4s/step - loss: 0.1553 - fbeta: 0.9116
Epoch 2/4
49/49 [==============================] - 182s 4s/step - loss: 0.1348 - fbeta: 0.9234
Epoch 3/4
49/49 [==============================] - 182s 4s/step - loss: 0.1234 - fbeta: 0.9298
Epoch 4/4
49/49 [==============================] - 182s 4s/step - loss: 0.1171 - fbeta: 0.9334


In [38]:
K.set_value(adam.lr, 0.001)
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=3,
                          shuffle=True)

Epoch 1/3
49/49 [==============================] - 185s 4s/step - loss: 0.1364 - fbeta: 0.9242
Epoch 2/3
49/49 [==============================] - 181s 4s/step - loss: 0.1171 - fbeta: 0.9353
Epoch 3/3
49/49 [==============================] - 182s 4s/step - loss: 0.0951 - fbeta: 0.9488


In [39]:
K.set_value(adam.lr, 0.001)
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=3,
                          shuffle=True)

Epoch 1/3
49/49 [==============================] - 185s 4s/step - loss: 0.0846 - fbeta: 0.9553
Epoch 2/3
49/49 [==============================] - 182s 4s/step - loss: 0.0736 - fbeta: 0.9612
Epoch 3/3
49/49 [==============================] - 182s 4s/step - loss: 0.0710 - fbeta: 0.9626


In [41]:
K.set_value(adam.lr, 0.001)
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=2,
                          shuffle=True)

Epoch 1/2
49/49 [==============================] - 188s 4s/step - loss: 0.0623 - fbeta: 0.9680
Epoch 2/2
49/49 [==============================] - 182s 4s/step - loss: 0.0588 - fbeta: 0.9700


In [43]:
K.set_value(adam.lr, 0.001)
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=2,
                          shuffle=True)

Epoch 1/2
49/49 [==============================] - 182s 4s/step - loss: 0.0575 - fbeta: 0.9712
Epoch 2/2
49/49 [==============================] - 175s 4s/step - loss: 0.0521 - fbeta: 0.9743


In [44]:
K.set_value(adam.lr, 0.001)
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=2,
                          shuffle=True)

Epoch 1/2
49/49 [==============================] - 178s 4s/step - loss: 0.0515 - fbeta: 0.9750
Epoch 2/2
49/49 [==============================] - 177s 4s/step - loss: 0.0464 - fbeta: 0.9777


In [47]:
K.set_value(adam.lr, 0.0001)
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=2,
                          shuffle=True)

Epoch 1/2
49/49 [==============================] - 178s 4s/step - loss: 0.0341 - fbeta: 0.9835
Epoch 2/2
49/49 [==============================] - 177s 4s/step - loss: 0.0311 - fbeta: 0.9846


In [48]:
K.set_value(adam.lr, 0.0001)
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=2,
                          shuffle=True)

Epoch 1/2
49/49 [==============================] - 180s 4s/step - loss: 0.0274 - fbeta: 0.9865
Epoch 2/2
49/49 [==============================] - 177s 4s/step - loss: 0.0264 - fbeta: 0.9869


In [50]:
K.set_value(adam.lr, 0.0001)
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=2,
                          shuffle=True)

Epoch 1/2
49/49 [==============================] - 179s 4s/step - loss: 0.0236 - fbeta: 0.9886
Epoch 2/2
49/49 [==============================] - 177s 4s/step - loss: 0.0242 - fbeta: 0.9882


In [52]:
K.set_value(adam.lr, 0.00001)
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=2,
                          shuffle=True)

Epoch 1/2
49/49 [==============================] - 179s 4s/step - loss: 0.0222 - fbeta: 0.9896
Epoch 2/2
49/49 [==============================] - 176s 4s/step - loss: 0.0214 - fbeta: 0.9895


In [53]:
save_model(model_final,'dl3-2')

Saved model to disk


### PREDICTIONG / SUBMISSION

##### LOAD MODEL AND WEIGHTS

In [54]:
gc.collect()

26799

In [55]:
%%time
image_lr=[]
test_img=[]
for i in list(test.Image_name):
    image_lr.append(read_image('./test_img/'+str(i),224,224))
test_img=np.array(image_lr,dtype=np.float32)
test_img=preprocess_input(test_img)

CPU times: user 1min 31s, sys: 724 ms, total: 1min 32s
Wall time: 1min 31s


In [ ]:
model_final=load_model('./weights/dl3-1')
batch_size=256

In [ ]:
test_datagen = ImageDataGenerator(
#rescale = 1./255,
horizontal_flip = True,
zoom_range = 0.2,
width_shift_range = 0.2,
height_shift_range=0.2,
rotation_range=20)



In [56]:
#test_generator = test_datagen.flow(test_img,batch_size = batch_size,shuffle=True)                                      
#pred=model_final.predict_generator(test_generator,verbose=1)
pred=model_final.predict(test_img,batch_size,verbose=1)


5400/5400 [==============================] - 67s 12ms/step


In [57]:
test.shape,test_img.shape

((5400, 1), (5400, 224, 224, 3))

In [58]:
pred.shape

(5400, 85)

In [59]:
sam=pd.read_csv('./meta-data/sample_submission.csv')
sam['Image_name']=test.Image_name
sam.iloc[:,1:]=pred
sam.head()

,Image_name,attrib_01,attrib_02,attrib_03,attrib_04,attrib_05,attrib_06,attrib_07,attrib_08,attrib_09,...,attrib_76,attrib_77,attrib_78,attrib_79,attrib_80,attrib_81,attrib_82,attrib_83,attrib_84,attrib_85
0,Image-1.jpg,6.590611e-09,2.707882e-21,9.575083e-20,1.167162e-11,1.268463e-19,0.999997,3.674178e-07,1.447827e-08,1.000000e+00,...,1.011168e-08,1.902872e-10,6.070585e-08,1.000000,9.999994e-01,2.497725e-08,0.000000e+00,0.999999,2.643347e-15,9.999990e-01
1,Image-2.jpg,7.526497e-03,8.639374e-03,1.139358e-05,9.765449e-01,7.089620e-04,0.996276,9.979795e-01,2.842009e-05,2.003714e-02,...,9.203355e-03,9.965774e-01,3.278466e-03,0.984282,1.455645e-02,4.313605e-05,1.387155e-04,0.009571,9.950004e-01,9.972405e-01
2,Image-3.jpg,1.594288e-04,8.821126e-10,3.041718e-08,9.963211e-01,7.202456e-12,0.000772,9.999815e-01,1.772646e-04,1.904662e-05,...,9.992078e-01,8.218395e-05,1.576865e-06,0.999880,1.371768e-08,2.738316e-07,3.635395e-12,0.999019,1.118150e-05,9.999999e-01
3,Image-4.jpg,9.993438e-01,4.352959e-06,1.002249e-04,8.755418e-01,2.154181e-01,0.999337,1.942482e-01,1.693443e-05,1.857406e-03,...,9.278719e-01,9.988418e-01,9.991425e-01,0.997216,1.700524e-04,1.328669e-03,3.556960e-14,0.707153,3.011992e-06,1.036977e-01
4,Image-5.jpg,2.638847e-12,7.089736e-16,8.960608e-20,9.999992e-01,5.148360e-14,0.999999,8.192628e-07,1.033152e-17,7.100001e-14,...,2.225923e-10,1.708061e-14,1.051992e-15,1.000000,9.999912e-01,7.622722e-19,5.697673e-32,1.000000,1.737507e-16,7.790074e-08


In [ ]:
sam=pd.read_csv('sub-xception-4-prob.csv')

In [60]:
temp=sam.copy()
for i in sam.iloc[:,1:]:
    temp[i]=sam[i].apply(lambda x: 1 if x>0.5 else 0)
        

In [61]:
temp.head()

,Image_name,attrib_01,attrib_02,attrib_03,attrib_04,attrib_05,attrib_06,attrib_07,attrib_08,attrib_09,...,attrib_76,attrib_77,attrib_78,attrib_79,attrib_80,attrib_81,attrib_82,attrib_83,attrib_84,attrib_85
0,Image-1.jpg,0,0,0,0,0,1,0,0,1,...,0,0,0,1,1,0,0,1,0,1
1,Image-2.jpg,0,0,0,1,0,1,1,0,0,...,0,1,0,1,0,0,0,0,1,1
2,Image-3.jpg,0,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,0,1,0,1
3,Image-4.jpg,1,0,0,1,0,1,0,0,0,...,1,1,1,1,0,0,0,1,0,0
4,Image-5.jpg,0,0,0,1,0,1,0,0,0,...,0,0,0,1,1,0,0,1,0,0


In [ ]:
sam.to_csv('sub-xception-4-prob.csv',index=False)

In [62]:

temp.to_csv('sub-xception-new-1.csv',index=False)
temp.shape

(5400, 86)